In [2]:
import pandas as pd
import re
import nltk
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter
from nltk.util import ngrams
import seaborn as sns
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
import gensim
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [1]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 12.4 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1


# **Loading Dataset**

In [4]:
url = "/content/IMDB Dataset.csv"
df = pd.read_csv(url)
print(df.head())

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


# **Cleaning Review**

In [5]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()
if 'review' not in df.columns:
    raise KeyError("The dataset does not contain a 'review' column. Please check the column names.")

# Function to tokenize text
def tokenize_text(text):
    words = re.findall(r'\b\w+\b', text)
    return words

# Function to clean and preprocess text
def preprocess_text(text):
    if not isinstance(text, str):
        return ""
    # Remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)
    # Remove special characters and digits
    text = re.sub(r'[^A-Za-z\s]', '', text)
    # Lowercasing
    text = text.lower()
    # Tokenization
    words = tokenize_text(text)
    # Stopword removal, stemming, and lemmatization
    words = [lemmatizer.lemmatize(ps.stem(word)) for word in words if word not in stop_words]
    return ' '.join(words)
df['cleaned_review'] = df['review'].apply(preprocess_text)
print(df[['review', 'cleaned_review']].head())

                                              review  \
0  One of the other reviewers has mentioned that ...   
1  A wonderful little production. <br /><br />The...   
2  I thought this was a wonderful way to spend ti...   
3  Basically there's a family where a little boy ...   
4  Petter Mattei's "Love in the Time of Money" is...   

                                      cleaned_review  
0  one review mention watch oz episod youll hook ...  
1  wonder littl product br br film techniqu unass...  
2  thought wonder way spend time hot summer weeke...  
3  basic there famili littl boy jake think there ...  
4  petter mattei love time money visual stun film...  


# **Mapping Sentiment**

In [6]:
def map_sentiment(sentiment):
    return 1 if sentiment == "positive" else 0
df['sentiment'] = df['sentiment'].apply(map_sentiment)


# **Training ML Models**

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df['cleaned_review'], df['sentiment'], test_size=0.2, random_state=42)

In [8]:
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [9]:
def train_and_evaluate(model, model_name):
    model.fit(X_train_tfidf, y_train)
    y_pred = model.predict(X_test_tfidf)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print(f"{model_name} - Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}")

 **LogisticRegression**

In [10]:
lr_model = LogisticRegression()
train_and_evaluate(lr_model, "Logistic Regression")

Logistic Regression - Accuracy: 0.8853, F1-score: 0.8879


**Naive Bayes**

In [11]:
nb_model = MultinomialNB()
train_and_evaluate(nb_model, "Naïve Bayes")

Naïve Bayes - Accuracy: 0.8469, F1-score: 0.8489


# **Predict Sentiment**

In [20]:
def predict_sentiment(review):
    original_review = review
    review = preprocess_text(review)
    review_tfidf = vectorizer.transform([review])
    prediction = lr_model.predict(review_tfidf)[0]
    sentiment = "Positive" if prediction == 1 else "Negative"
    print(f"Review: {original_review}\nPredicted Sentiment: {sentiment}")




**User Input**

In [21]:
user_review = "This movie was fantastic! The acting was great."
predict_sentiment(user_review)

Review: This movie was fantastic! The acting was great.
Predicted Sentiment: Positive
